# 15.10 Sudoku Solver Takeaway

In order to solve this with backtracking recursion, we need to go through each empty index starting from (0, 0) and run a solve_partial_sudoku recursive function.  
If it's empty, we need to try numbers 1-9, checking if these 3 conditions are met:
- No other numbers in the row are the same
- No other numbers in the column are the same
- No other numbers in the subgrid are the same
These 3 checks will be encapsulated in a valid_to_add() function


If  the numer is *valid_to_add()* we need to recurse from here, and go to the next empty index


[Video Explanation](https://www.youtube.com/watch?v=eAFcj_2quWI) in case this text is too confusing.

## The hard condition: checking current subgrid

We need to get the subgrid's size by just doing a squareroot, since we know it's a nxn grid.

Then we can locate the specific region by integer dividing the location i and j by the subgrid's size

Afterwards, we can make slick use of itertools.product to get all the combinations of elements (or indices) in the subgrid, and check if any already hold the value in our sudoku example in a single **if any()** statement

In [22]:
import math
import itertools

# Example sudoku list
sudoku_assignment = [[0, 3, 2, 0, 0, 0, 8, 0, 4], [8, 0, 0, 2, 0, 0, 0, 7, 0], [0, 1, 7, 0, 0, 5, 9, 0, 6], [5, 8, 0, 0, 2, 0, 0, 3, 0], [0, 0, 6, 0, 4, 0, 7, 0, 0], [0, 0, 4, 9, 1, 3, 0, 6, 0], [0, 0, 0, 7, 3, 0, 2, 0, 0], [0, 5, 9, 0, 0, 0, 0, 0, 1], [1, 0, 0, 8, 0, 9, 0, 0, 0]]

def checksubgrid(i, j, value):
    region_size = int(math.sqrt(len(sudoku_assignment)))
    # I and J are selecting which of the 9 subgrids, from (0, 0) to (2, 2) we are checking
    I = i // region_size
    J = j // region_size
    if any(value == sudoku_assignment[region_size * I + a][region_size * J + b] for a, b in itertools.product(range(region_size), repeat=2)):
        return False
    return True    

value = 6
result = checksubgrid(0, 0, value)
if result:
    print("We can add %s to the subgrid" % value)
else:
    print("%s already exists in this subgrid" % value)


print("How itertools.product works")
for a, b in itertools.product([0, 1, 2], repeat=2):
        print(a, b)

We can add 6 to the subgrid
How itertools.product works
0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2


# Full Answer

In [ ]:
import math
import itertools

def solve_sudoku(partial_assignment: List[List[int]]) -> bool:
    def solve_partial_sudoku(i, j):
        if i == len(partial_assignment):
            i = 0  # Starts a row.
            j += 1
            if j == len(partial_assignment[i]):
                return True  # Entire matrix has been filled without conflict.

        # Skips nonempty entries.
        if partial_assignment[i][j] != empty_entry:
            return solve_partial_sudoku(i + 1, j)

        def valid_to_add(i, j, val):
            # Check row constraints.
            if any(val == partial_assignment[k][j] for k in range(len(partial_assignment))):
                return False

            # Check column constraints.
            if val in partial_assignment[i]:
                return False

            # Check region constraints.
            region_size = int(math.sqrt(len(partial_assignment)))
            I = i // region_size
            J = j // region_size
            return not any(
                val == partial_assignment[region_size * I +
                                          a][region_size * J + b]
                for a, b in itertools.product(range(region_size), repeat=2))

        for val in range(1, len(partial_assignment) + 1):
            # It's substantially quicker to check if entry val with any of the
            # constraints if we add it at (i,j) adding it, rather than adding it and
            # then checking all constraints. The reason is that we know we are
            # starting with a valid configuration, and the only entry which can
            # cause a problem is entry val at (i,j).
            if valid_to_add(i, j, val):
                partial_assignment[i][j] = val
                if solve_partial_sudoku(i + 1, j):
                    return True

        # If none of the plays work, reset to empty as we jump out of each solve_partial_sudoku
		# THIS is the essence of backtracking!! we are covering our tracks for the next attempt so the board is reset
        partial_assignment[i][j] = empty_entry  # Undo assignment.
        return False

    empty_entry = 0
    return solve_partial_sudoku(0, 0)